### <b> Bibliotecas utilizadas para realizar a instalação: </b>

* pip install pyaudio // conda install -c anaconda pyaudio <br>
* pip install gtts <br>
* pip install soundfile <br>
* pip install sounddevice <br>
* pip install SpeechRecognition <br>
* pip install google.generativeai

### <b> Passo a passo de cada etapa </b>

In [61]:
import os
import time
import textwrap
import soundfile as sf
import sounddevice as sd
import speech_recognition as sr
import google.generativeai as genai 

from gtts import gTTS
from playsound import playsound
from IPython.display import display
from IPython.display import Markdown
from dotenv import load_dotenv

load_dotenv()

# Configurar para o texto ficar de forma mais organizada.
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

##### <b> Configurar a variável de ambiente criando um arquivo .env </b>

* Verificar: https://ai.google.dev/gemini-api/docs/get-started/python

In [2]:
genai.configure(api_key=os.getenv('GOOGLE_API_KEY'))

##### <b> Configurar o modelo conforme a sua necessidade </b>

In [3]:
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 0
}

safety_settings = [
    {
        "category": "HARM_CATEGORY_DANGEROUS",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_HARASSMENT",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_HATE_SPEECH",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
        "threshold": "BLOCK_NONE",
    },
]

model = genai.GenerativeModel(model_name='gemini-1.0-pro',
                                  generation_config=generation_config,
                                  safety_settings=safety_settings,)

#### <b> Código para captar a fala do usuário no microfone. </b>

*  Obs.: Ativar o microfone do computador

In [ ]:
record = sr.Recognizer()
microphone = sr.Microphone()

with microphone as fonte_audio:
    print("Fale o que gostaria...")
    audio = record.listen(fonte_audio)

speech = record.recognize_google(audio, language="pt-BR")
print("O usuário disse:", speech)

##### Passar o texto salvo da gravação da fala do usuário para o modelo.

In [5]:
response = model.generate_content(speech)

##### Utilizando o markdown dessa forma no Macbook para que o texto fique mais organizado ao usuário.

In [ ]:
for candidate in response.candidates:
    response_speech = " ".join(part.text.replace('**', '') for part in candidate.content.parts)
    response_text_to_markdown = to_markdown(response_speech)
    display(response_text_to_markdown)

##### Transforma o texto em áudio. Salva no repositório e reproduz para o usuário.

In [72]:
text_to_audio = gTTS(response_speech, lang='pt-br')
text_to_audio.save('/Users/pesati/Desktop/Imersao Alura/audio/speech.mp3')
playsound('/Users/pesati/Desktop/Imersao Alura/audio/speech.mp3')

# Chatbot com a IA do Google

In [101]:
playsound('/Users/pesati/Desktop/Imersao Alura/audio/finish.mp3')
wait_user = '/Users/pesati/Desktop/Imersao Alura/audio/wait_user.mp3'
speech_user = '/Users/pesati/Desktop/Imersao Alura/audio/speech.mp3'
url = '/Users/pesati/Desktop/Imersao Alura/audio/speech.mp3'

record = sr.Recognizer()
microphone = sr.Microphone()

In [ ]:
chat = model.start_chat(history=[])

while speech != "encerrar":
    
    with microphone as fonte_audio:
        print("Fale o que gostaria...")
        playsound(wait_user)
        audio = record.listen(fonte_audio)
    
    speech = record.recognize_google(audio, language="pt-BR")
    print("O usuário disse:", speech + "\n")
    
    response = chat.send_message(speech)
    
    if(speech == 'encerrar'):
        print("Encerrando o chat!")
        break
    
    response_speech = response.candidates[0].content.parts[0].text
    response_text_to_markdown = to_markdown(response_speech)
    display(response_text_to_markdown)
    print('-------------------------------------------')
    
    text_to_audio = gTTS(response_speech, lang='pt-br')
    text_to_audio.save(url)
    playsound(speech_user)
    
    chat.rewind()
    
    time.sleep(5)
